In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import os
import time

print("🚀 मॉडल को 'Staging' से 'Production' में प्रमोट करने की प्रक्रिया शुरू हो रही है।")

# Unity Catalog के लिए MLflow registry URI सेट करें
try:
    if "DATABRICKS_RUNTIME_VERSION" in os.environ:
        mlflow.set_registry_uri("databricks-uc")
        print("✅ MLflow रजिस्ट्री URI को यूनिटी कैटलॉग पर सेट किया गया।")
    else:
        print("⚠️ Databricks वातावरण में नहीं चल रहा है। URI सेट करना छोड़ दिया गया है।")
        
    client = MlflowClient()
    print("✅ MLflow क्लाइंट सफलतापूर्वक इनिशियलाइज़ किया गया।")
except Exception as e:
    print(f"❌ MLflow क्लाइंट इनिशियलाइज़ेशन विफल हुआ: {e}")
    exit()

# यूनिटी कैटलॉग में आपके मॉडल का पूरा नाम और एलियास
model_name = "workspace.ml.house_price_model"
staging_alias = "Staging"
production_alias = "Production"

# Staging एलियास के साथ नवीनतम मॉडल संस्करण प्राप्त करें
try:
    # Staging alias से model version प्राप्त करें
    staging_version = client.get_model_version_by_alias(model_name, staging_alias)
    version_number = staging_version.version
    print(f"ℹ️ '{staging_alias}' पर नवीनतम मॉडल संस्करण मिला: v{version_number}")

    # मॉडल की स्थिति 'READY' होने तक प्रतीक्षा करें
    status = staging_version.status
    max_wait_time = 300  # अधिकतम 5 मिनट
    elapsed_time = 0
    check_interval = 5

    while status != "READY" and elapsed_time < max_wait_time:
        print(f"⏳ मॉडल v{version_number} अभी '{status}' स्थिति में है। {check_interval} सेकंड के लिए प्रतीक्षा कर रहा हूँ...")
        time.sleep(check_interval)
        elapsed_time += check_interval
        staging_version = client.get_model_version(model_name, version_number)
        status = staging_version.status

    if status == "READY":
        print(f"✅ मॉडल v{version_number} अब '{status}' स्थिति में है।")

        # सुनिश्चित करें कि प्रोडक्शन में पहले से कोई अन्य मॉडल न हो
        try:
            old_prod_version = client.get_model_version_by_alias(model_name, production_alias)
            print(f"🔄 पुराने '{production_alias}' मॉडल को v{old_prod_version.version} से हटा रहा हूँ।")
            client.delete_model_version_alias(model_name, production_alias)
            print("✅ पुराना Production alias हटा दिया गया।")
        except Exception:
            print(f"ℹ️ कोई मौजूदा '{production_alias}' मॉडल नहीं मिला।")

        # मॉडल को 'Production' एलियास पर सेट करें
        client.set_registered_model_alias(model_name, production_alias, version_number)
        
        print(f"✅ मॉडल संस्करण v{version_number} को सफलतापूर्वक '{production_alias}' एलियास पर प्रमोट किया गया।")
        
        # Verification करें
        try:
            prod_version = client.get_model_version_by_alias(model_name, production_alias)
            if prod_version.version == version_number:
                print(f"🎯 Verification: Model v{version_number} successfully promoted to Production!")
            else:
                print("⚠️ Warning: Production promotion में verification issue है।")
        except Exception as verify_error:
            print(f"⚠️ Production verification में समस्या: {verify_error}")
            
    else:
        print(f"❌ मॉडल अभी भी '{status}' स्थिति में है या प्रतीक्षा समय समाप्त हो गया है। प्रमोट नहीं किया गया।")
        print("💡 सुझाव: MLflow UI में model की status manually check करें।")
        
except Exception as e:
    print(f"❌ मॉडल को प्रमोट करने में त्रुटि: {e}")
    print("\n🔍 डिबगिंग जानकारी:")
    print(f"1. मॉडल नाम: {model_name}")
    print(f"2. Staging alias: {staging_alias}")
    print(f"3. Production alias: {production_alias}")
    print("4. कृपया सुनिश्चित करें कि:")
    print("   - Staging alias properly set है")
    print("   - आपके पास model promote permissions हैं")
    print("   - Unity Catalog properly configured है")